In [1]:
%store -r data
%store -r columns

In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

In [3]:
raw_data = pd.DataFrame(data, columns = columns)

T_ARTIST

In [4]:
artists_names = []
id_artists = []
for i in range(len(raw_data['artists_track'])):
    for j in range(len(raw_data['artists_track'][i])):
        artists_names.append(raw_data['artists_track'][i][j])
        id_artists.append(raw_data['id_artist_track'][i][j])


In [5]:
unique_artists_names = pd.unique(artists_names)

In [6]:
unique_id_artists = pd.unique(id_artists)

In [7]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        
    except Error as e:
        print(e)

    return conn

In [8]:
connection = create_connection(r"D:\SQLITE\DB_SPOTIFY")

In [9]:
cur = connection.cursor()

In [10]:
query_id_spotify = '''

SELECT ID_SPOTIFY_ARTIST
FROM t_artists;

'''
query_id_max = '''
SELECT MAX(id_artist)
from t_artists
'''

In [11]:
cur.execute(query_id_max)
id_max = cur.fetchall()[0][0]
if id_max is None:
    id_max = 0
else:
    id_max = id_max+1

In [12]:
cur.execute(query_id_spotify)
rows = cur.fetchall()

In [13]:
id_artists_db = np.array(rows)

In [14]:
common_elements = np.intersect1d(id_artists_db,unique_id_artists)

In [15]:
positions = np.where(np.in1d(unique_id_artists, common_elements))

In [16]:
column_name = np.delete(unique_artists_names,positions)
column_id_artists_spotify = np.delete(unique_id_artists,positions)
column_id_artist = list(map(lambda i: id_max + i, range(len(column_id_artists_spotify))))
final_data_artists = list(zip(column_id_artist,column_name,column_id_artists_spotify))

In [17]:
t_artists = pd.DataFrame(final_data_artists,columns = ['ID_ARTIST','NAME','ID_SPOTIFY_ARTIST'])

In [18]:
t_artists.to_sql('T_ARTISTS', connection, if_exists='append',index = False)

5

T_ALBUMS

In [19]:
unique_albums = raw_data[['name_album','release_date_album',
                          'total_tracks_album','id_album','artists_names_album', 
                          'id_artists_album']].drop_duplicates(subset = 'id_album').reset_index(drop = True)
unique_albums = unique_albums.rename(columns = {'name_album': 'NAME','release_date_album':'RELEASE_DATE',
                                                'total_tracks_album':'TOTAL_TRACKS','id_album':'ID_SPOTIFY_ALBUM',
                                                'id_artists_album':'ID_SPOTIFY_ARTIST'})

In [20]:
t_albums = pd.merge(right = pd.read_sql('SELECT ID_ARTIST, ID_SPOTIFY_ARTIST FROM T_ARTISTS',connection),
         left = unique_albums,
         on = 'ID_SPOTIFY_ARTIST',
         how = 'inner')
t_albums = t_albums.drop(columns = ['artists_names_album','ID_SPOTIFY_ARTIST'])

In [21]:
query_id_spotify = '''

SELECT ID_SPOTIFY_ALBUM
FROM T_ALBUMS;

'''
query_id_max = '''
SELECT MAX(id_album)
from T_ALBUMS
'''

In [22]:
cur.execute(query_id_max)
id_max = cur.fetchall()[0][0]
if id_max is None:
    id_max = 0
else:
    id_max = id_max+1

In [23]:
cur.execute(query_id_spotify)
rows = cur.fetchall()

In [24]:
id_album_db = np.array(rows)

In [25]:
unique_id_album = t_albums['ID_SPOTIFY_ALBUM'].to_numpy()

In [26]:
common_elements = np.intersect1d(id_album_db,unique_id_album)

In [27]:
positions = np.where(np.in1d(unique_id_album, common_elements))

In [28]:
t_albums = t_albums.drop(t_albums.index[positions])

In [29]:
t_albums['ID_ALBUM'] = list(map(lambda i: id_max + i, range(len(t_albums))))

In [30]:
t_albums.to_sql('T_ALBUMS', connection, if_exists='append',index = False)

5

T_MUSICS_HAS_T_ARTISTS & T_MUSIC

In [31]:
unique_musics = raw_data[['name_track','duration_ms_track','track_number_track',
          'explicit_track','id_track','id_album',
          'id_artist_track']].drop_duplicates(subset = 'id_track').reset_index(drop = True).explode('id_artist_track')

In [32]:
unique_musics = pd.merge(right = pd.read_sql('SELECT ID_ARTIST, ID_SPOTIFY_ARTIST FROM T_ARTISTS',connection),
         left = unique_musics,
         right_on = 'ID_SPOTIFY_ARTIST',
         left_on = 'id_artist_track',
         how = 'inner').drop(columns = ['ID_SPOTIFY_ARTIST','id_artist_track'])

In [33]:
unique_musics = pd.merge(right = pd.read_sql('SELECT ID_ALBUM, ID_SPOTIFY_ALBUM FROM T_ALBUMS',connection),
                   left = unique_musics,
                   right_on = 'ID_SPOTIFY_ALBUM',
                   left_on = 'id_album').drop(columns = ['id_album','ID_SPOTIFY_ALBUM']).rename(
                                                        columns = {'name_track': 'NAME_MUSIC',
                                                                   'duration_ms_track':'DURATION_MS',
                                                                   'track_number_track':'TRACK_NUMBER',
                                                                   'explicit_track':'EXPLICIT',
                                                                   'id_track':'ID_SPOTIFY_MUSIC'})

In [34]:
query_id_spotify = '''

SELECT ID_SPOTIFY_MUSIC
FROM T_MUSICS;

'''
query_id_max = '''
SELECT MAX(ID_MUSIC)
from T_MUSICS;
'''

In [35]:
cur.execute(query_id_max)
id_max = cur.fetchall()[0][0]
if id_max is None:
    id_max = 0
else:
    id_max = id_max+1

In [36]:
cur.execute(query_id_spotify)
rows = cur.fetchall()

In [37]:
id_music_db = np.array(rows)

In [38]:
unique_id_music = unique_musics['ID_SPOTIFY_MUSIC'].to_numpy()

In [39]:
common_elements = np.intersect1d(id_music_db,unique_id_music)

In [40]:
positions = np.where(np.in1d(unique_id_music, common_elements))

In [41]:
unique_musics = unique_musics.drop(unique_musics.index[positions])

In [42]:
id_new_music = list(map(lambda i: id_max + i, range(len(unique_musics.drop_duplicates('ID_SPOTIFY_MUSIC')))))

In [43]:
t_musics_has_t_artists =unique_musics[['ID_ARTIST','ID_SPOTIFY_MUSIC']]

In [44]:
unique_musics = unique_musics.drop_duplicates('ID_SPOTIFY_MUSIC').reset_index(drop =True).drop(columns ='ID_ARTIST')

In [45]:
unique_musics['ID_MUSIC'] = id_new_music

In [46]:
unique_musics.to_sql('T_MUSICS', connection, if_exists='append',index = False)

9

In [47]:
t_musics_has_t_artists = pd.merge(left = pd.read_sql('SELECT ID_MUSIC, ID_SPOTIFY_MUSIC FROM T_MUSICS',connection),
         right = t_musics_has_t_artists,
         on = 'ID_SPOTIFY_MUSIC',
         how = 'inner').drop(columns =['ID_SPOTIFY_MUSIC'])

In [48]:
t_musics_has_t_artists.to_sql('T_MUSICS_HAS_T_ARTISTS', connection, if_exists='append',index = False)

9

T_EXECUTED

In [49]:
raw_data.columns

Index(['played_at', 'id_track', 'name_track', 'explicit_track',
       'duration_ms_track', 'track_number_track', 'id_artist_track',
       'artists_track', 'id_album', 'name_album', 'release_date_album',
       'total_tracks_album', 'artists_names_album', 'id_artists_album'],
      dtype='object')

In [50]:
t_executed = pd.merge(left = raw_data[['played_at','id_track']],
         right = pd.read_sql('SELECT ID_MUSIC, ID_SPOTIFY_MUSIC FROM T_MUSICS',connection),
         left_on = 'id_track',
         right_on = 'ID_SPOTIFY_MUSIC',
         how = 'inner').sort_values(by = ['played_at'], ascending= 1).drop(columns = ['id_track','ID_SPOTIFY_MUSIC']).rename(
                                                        columns = {'played_at': 'PLAYED_AT'})

In [51]:
query_dt_max = '''
SELECT MAX(PLAYED_AT)
from T_EXECUTED;
'''

In [52]:
cur.execute(query_dt_max)
dt_max = cur.fetchall()[0][0]
if dt_max is None:
    dt_max = '1998-12-21'
else:
    pass

In [53]:
t_executed = t_executed.loc[t_executed['PLAYED_AT']>dt_max]

In [54]:
t_executed.to_sql('T_EXECUTED', connection, if_exists='append',index = False)

11

In [55]:
connection.close()